# * Create a sqlite database for tables using using SQLAlchemy.
# * Clean measurements and stations CSV data using pandas.
# * Create a database called hawaii.sqlite after creating engine, connection  to connect to Database. 
# * Create ORM classes for each table using declarative_base.
# * Classes created: a) Measurement b) Station.
# * Make sure to define your primary keys.
# * Create the tables in the database using create_all.

In [1]:
# Dependencies from pandas
import pandas as pd
import os

In [2]:
# SQLAlchemy dependencies to create engine & declarative base 
from sqlalchemy import create_engine ,inspect, MetaData
from sqlalchemy.orm import Session


from sqlalchemy.ext.declarative import declarative_base

# For data types
from sqlalchemy import Column, Integer, String, Float, Date

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb

<function pymysql.install_as_MySQLdb()>

In [3]:
# Create object for the default declarative base
Base = declarative_base()

In [4]:
# Create Base Classes :- Measurements and Stations
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key = True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    
class Stations(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key = True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

# Reflect Tables into SQLAlchemy ORM

In [5]:
# Create SQlite engine
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()


In [6]:
# Add metadata layer to tables
Base.metadata.create_all(engine)

In [7]:
# Create a Session Object to temporary bind and Connect to DB
session = Session(bind=engine)

In [8]:
# Creating the filepaths# Creat 
filepath1=os.path.join("ProcessedData","pro_hawaii_measurements.csv")
filepath2=os.path.join("ProcessedData","pro_hawaii_stations.csv")

# Read the csv files and convert to dataframe
Hawaii_Measurement=pd.read_csv(filepath1)
print("Completed Reading File:%s"%filepath1)
Hawaii_Station=pd.read_csv(filepath2)
print("Completed Reading File:%s"%filepath2)

Completed Reading File:ProcessedData\pro_hawaii_measurements.csv
Completed Reading File:ProcessedData\pro_hawaii_stations.csv


In [9]:
# Drop table if it had been created before.
try:
    conn.execute("DROP TABLE Measurements")
    print("Dropping Table: Measurements as it already exists")
    conn.execute("DROP TABLE Stations")
    print("Dropping Table: Stations as it already exists")
except:
    pass

Dropping Table: Measurements as it already exists
Dropping Table: Stations as it already exists


In [10]:
# Data from CSVs to respective tables
Hawaii_Measurement.to_sql('measurements', engine, if_exists = 'append', index = True)
Hawaii_Station.to_sql('stations',engine, if_exists = 'append', index = True)
session.commit()

In [11]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Check if the table has been created within the database
inspector.get_table_names()

['measurements', 'stations']

In [12]:
# Printing the first 10 rows of Measurement table
conn.execute("SELECT * FROM Measurements LIMIT(10)").fetchall()

[(0, 'USC00519397', '2010-01-01', 0.08, 65),
 (1, 'USC00519397', '2010-01-02', 0.0, 63),
 (2, 'USC00519397', '2010-01-03', 0.0, 74),
 (3, 'USC00519397', '2010-01-04', 0.0, 76),
 (4, 'USC00519397', '2010-01-06', 0.0, 73),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64)]

In [13]:
# Printing the first 10 rows of Station table
conn.execute("SELECT * FROM Stations LIMIT(10)").fetchall()

[(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (5, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (6, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (7, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [14]:
# Close the connection established
conn.close()